In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import warnings
import geopandas


In [ ]:
national_df = pd.read_csv('../Resources/national_Heart_Disease.csv')
national_df.head()

In [ ]:
Overall_df = national_df[national_df['Gender'] == 'Overall']
Overall_df = Overall_df.sort_values(by='Data Value (Per 100,000 Population)', ascending=False)
Overall_df = Overall_df[Overall_df['Race/Ethnicity'] != 'Overall']

In [ ]:
plt.style.available

In [ ]:
# plt.style.use('grayscale')
plt.style.use('seaborn-v0_8-poster')


In [ ]:
data = Overall_df["Data Value (Per 100,000 Population)"]
labels = Overall_df["Race/Ethnicity"]
explode = [0.07,0,0,0,0,0,0]

# colors = ["#3A8F9D", "#FF6F61", "#FF7F50", "#FF8C69", "#FFB74D", "#FFCC80", "#FFE0B2"]
colors = ["#3A8F9D", "#00B2B2", "#00B2B5", "#00C2C9", "#3BC6D9", "#6FD0E1", "#A0D8E6"]

plt.figure(figsize=(6,8))
plt.title("United States Heart Disease by Ethnicity", fontsize=18, fontweight="bold")
plt.pie(data, explode=explode, labels=labels, colors=colors, autopct= '%1.1f%%', textprops={'fontsize': 16}, pctdistance=0.75, labeldistance=1.1, startangle=120)
plt.axis('equal')
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(8, 6))

# Create a horizontal bar plot
sns.barplot(data=Overall_df, x='Data Value (Per 100,000 Population)', y='Race/Ethnicity', color='b')

# Customize the plot
plt.xlabel('Mortality Rate (Per 100,000 Population)', fontsize=16)
plt.ylabel('Ethnicity', fontsize=16)
plt.title('United States Heart Disease by Ethnicity', fontsize=16, fontweight="bold",)

# Show the plot
plt.tight_layout()
plt.show()

# Plotly Chloropleth

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.colors as colors

In [ ]:
state_df = pd.read_csv('../Resources/state_Heart_Disease.csv')


In [ ]:
state_df = state_df[state_df['Race/Ethnicity'] == 'Overall']


In [ ]:
state_df_pivoted = state_df.pivot(index='State', columns='Gender', values='Data Value (Per 100,000 Population)').reset_index()
state_df_pivoted.rename(columns={'Overall': 'Overall Rate', 'Male': 'Male Rate', 'Female': 'Female Rate'}, inplace=True)
state_df_pivoted.head()

In [ ]:
state_full_names = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
    'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
    'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
    'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
    'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
    'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
    'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
    'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
    'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
    'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
    'WI': 'Wisconsin', 'WY': 'Wyoming', 
}

state_df_pivoted['Full State Name'] = state_df_pivoted['State'].map(state_full_names)
state_df_pivoted["M/F Difference"] = state_df_pivoted['Male Rate'] - state_df_pivoted['Female Rate']

In [ ]:
states_to_remove = ['GU', 'DC', 'PR', 'VI', 'AS', 'MP']
state_df_pivoted = state_df_pivoted[~state_df_pivoted['State'].isin(states_to_remove)]

In [ ]:
fig = go.Figure(data=go.Choropleth(
    locations=state_df_pivoted['State'],
    z=state_df_pivoted['Overall Rate'],
    locationmode='USA-states',
    colorscale='Magma_r',
    zmin=200, 
    zmax=600,  
    autocolorscale=False,
    marker_line_color='white',
    marker_line_width=0.5,
    colorbar_title="Mortality Rate (Per 100,000)",
    hoverinfo='text',
    customdata=state_df_pivoted[['Full State Name', 'Male Rate', 'Female Rate']],
    hovertemplate="<span style='font-size: 15px;'><b>%{customdata[0]}</b></span><br>" +  # Display the state name
                  "Mortality Rate: %{z}<br>" +               # Display the overall mortality rate
                  # "Male Rate:     %{customdata[1]}<br>" +   # Display the male mortality rate
                  # "Female Rate: %{customdata[2]}<br>" + # Display the female mortality rate
                  "<extra></extra>",  # Remove the default hover text
))

fig.update_layout(
    title_text = "<b>USA Heart Disease Mortality Rate</b>",
    title_x=0.5,  # Center the title
    title_y=0.9, # Center the title
    geo_scope= 'usa',
    width = 1000,
    height = 600,
    showlegend=False,
    paper_bgcolor='White',
    title_font_color='Black',
    title_font_size=25,
    margin=dict(l=10, r=10, t=50, b=10)  # Adjust the margins (left, right, top, bottom)
)

fig.show()

In [ ]:
state_df_sorted = state_df_pivoted.sort_values(by='Overall Rate', ascending=False)
state_df_sorted.head()

In [ ]:
fig = px.scatter(state_df_pivoted, 
                 x='Overall Rate', 
                 y='M/F Difference', 
                 hover_name='Full State Name', 
                 size='Overall Rate', 
                 color='State', 
                 height=600)  # Set the height to your desired value (e.g., 600 pixels)
fig.update_traces(showlegend=False)
fig.update_layout(title='<b>USA Heart Disease Overall Mortality Rate vs Male/Female Difference Rate</b>')
fig.show()
# color_discrete_sequence=px.colors.sequential.Reds

# Dash App

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
state_df_pivoted.head()

In [ ]:
female_df = state_df_pivoted.sort_values(by='Female Rate', ascending=True)
female_df.head()

In [ ]:
app = dash.Dash(__name__)

In [ ]:
app.layout = html.Div([
    html.Div([
        html.Div('Select Gender:', style={'fontSize': 25, 'color': 'white'}),
        dcc.RadioItems(
            id='gender-selector',
            options=[
                {'label': 'Male', 'value': 'Male Rate'},
                {'label': 'Female', 'value': 'Female Rate'},
                {'label': 'Overall', 'value': 'Overall Rate'}
            ],
            value='Overall Rate',  # Default value
            labelStyle={'display': 'inline-block', 'margin': '10px', 'color': 'white'}  # Style for radio items
        )
    ], style={'textAlign': 'left', 'marginBottom': '10px'}),
    
    dcc.Graph(id='choropleth-map', style={'width': '100%', 'height': '400px'})  # Set the graph to full width
])

In [ ]:
@app.callback(
    Output('choropleth-map', 'figure'),
    Input('gender-selector', 'value')
)
def update_graphs(selected_rate):
    # Update Choropleth Map
    choropleth_fig = go.Figure(data=go.Choropleth(
        locations=state_df_pivoted['State'],
        z=state_df_pivoted[selected_rate],
        locationmode='USA-states',
        colorscale='Magma_r',
        zmin=160, 
        zmax=600,  
        autocolorscale=False,
        marker_line_color='white',
        marker_line_width=0.5,
        colorbar_title="Mortality Rate (Per 100,000)",
        hoverinfo='text',
        customdata=state_df_pivoted[['Full State Name', 'Male Rate', 'Female Rate']],
        hovertemplate="<span style='font-size: 14px;'><b>%{customdata[0]}</b></span><br>" +
                      "Mortality Rate: %{z}<br>" +
                      "<extra></extra>",
    ))

    choropleth_fig.update_layout(
        title_text="<b>USA Heart Disease Mortality Rate by Gender</b>",
        title_y=0.9,
        geo_scope='usa',
        width=1000,  # You can adjust this if needed
        height=600,  # Keep the height consistent
        showlegend=False,
        paper_bgcolor='darkgrey',
        title_font_color='White',
        title_font_size=28,  # Reduce title font size
        margin=dict(l=10, r=10, t=50, b=10)
    )

    return choropleth_fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

# Reading the data from a database

In [ ]:
# to read the csv into a database, we imported it using DB Browser